https://keras.io/examples/nlp/lstm_seq2seq/

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [2]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = "fra.txt"

In [3]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split("\t")
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0


Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 94
Max sequence length for inputs: 15
Max sequence length for outputs: 59


In [4]:
# Define an input sequence and process it.
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)


In [5]:
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)

Epoch 1/100
125/125 [==============================] - 23s 182ms/step - loss: 1.1795 - accuracy: 0.7268 - val_loss: 1.0661 - val_accuracy: 0.7216
Epoch 2/100
125/125 [==============================] - 21s 167ms/step - loss: 0.8501 - accuracy: 0.7703 - val_loss: 0.8285 - val_accuracy: 0.7674
Epoch 3/100
125/125 [==============================] - 25s 197ms/step - loss: 0.6745 - accuracy: 0.8094 - val_loss: 0.7118 - val_accuracy: 0.7931
Epoch 4/100
125/125 [==============================] - 23s 181ms/step - loss: 0.5881 - accuracy: 0.8294 - val_loss: 0.6553 - val_accuracy: 0.8047
Epoch 5/100
125/125 [==============================] - 22s 176ms/step - loss: 0.5380 - accuracy: 0.8431 - val_loss: 0.6198 - val_accuracy: 0.8174
Epoch 6/100
125/125 [==============================] - 22s 174ms/step - loss: 0.5006 - accuracy: 0.8532 - val_loss: 0.5920 - val_accuracy: 0.8256
Epoch 7/100
125/125 [==============================] - 22s 176ms/step - loss: 0.4724 - accuracy: 0.8604 - val_loss: 0.5638 -

Epoch 57/100
125/125 [==============================] - 25s 199ms/step - loss: 0.0949 - accuracy: 0.9699 - val_loss: 0.5840 - val_accuracy: 0.8718
Epoch 58/100
125/125 [==============================] - 24s 193ms/step - loss: 0.0926 - accuracy: 0.9708 - val_loss: 0.5960 - val_accuracy: 0.8711
Epoch 59/100
125/125 [==============================] - 24s 192ms/step - loss: 0.0907 - accuracy: 0.9713 - val_loss: 0.5955 - val_accuracy: 0.8708
Epoch 60/100
125/125 [==============================] - 25s 198ms/step - loss: 0.0886 - accuracy: 0.9717 - val_loss: 0.6022 - val_accuracy: 0.8703
Epoch 61/100
125/125 [==============================] - 25s 202ms/step - loss: 0.0867 - accuracy: 0.9723 - val_loss: 0.6093 - val_accuracy: 0.8707
Epoch 62/100
125/125 [==============================] - 24s 188ms/step - loss: 0.0848 - accuracy: 0.9730 - val_loss: 0.6098 - val_accuracy: 0.8711
Epoch 63/100
125/125 [==============================] - 24s 193ms/step - loss: 0.0837 - accuracy: 0.9734 - val_loss: 0